In [ ]:
# Load Base Model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

BASE_MODEL = "TBD"
LEARNING_RATE = 2e-5
MAX_LENGTH = 256
BATCH_SIZE = 16
EPOCHS = 20

id2label = #TODO: Define {0,1,2,.. : 1h,2h,4h,..} 
label2id = #TODO: inverted id2label

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
# classification:
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, id2label=id2label, label2id=label2id)
# regression:
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, num_labels=1)

In [ ]:
# Tokenize input
def tokenizer(df):
    label = df["y"] 
    df = tokenizer(df["X"], truncation=True, padding="max_length", max_length=256)
    # classification:
    df["y"] = label
    # regression
    df["y"] = float(label)
    return df

In [ ]:
# Prepare Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

PATH = "../" #TBD

# Load Dataset:
df = pd.read_csv(PATH)

X_train, X_test, y_train, y_test = train_test_split(df["X"], df["y"], test_size=0.20, random_state=123)

#TODO: Fix this, train & validation need to be of dim (X, 2)
ds = {"train": raw_train_ds, "test": raw_test_ds}

In [ ]:
# Training
from transformers import TrainingArguments
from transformers import Trainer
#TODO: Check why imports not working, huggingface API:
# https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
# https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer

training_args = TrainingArguments(
    output_dir="TBD",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="accuracy",
    load_best_model_at_end=True,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
# Evaluation

# classification:
trainer.evaluate()

# regression:
# TODO: Change evaluation metric from accuracy to MSE / RMSE / MAE for regression, if needed
# -> override "compute_metrics" parameter of Trainer